In [22]:
import pandas as pd
import numpy as np
import sqlalchemy
import datetime
from sql_functions import *
import dotenv
import psycopg2


importing the csv in df

In [31]:
df1=pd.read_csv('Data/my_sales_before_14010624.csv' , encoding='utf_16_le')
df1.to_csv('Data/sales_before_newenc.csv' , encoding='utf_8')

In [2]:
sales_before_df=pd.read_csv('Data/my_sales_before_14010624.csv' , encoding='utf_16_le')

In [3]:
sales_before_df

,TRANSACTION_ID,TRANSACTION_DATE,CUSOMER_ID,PRODUCT_TYPE,PRODUCTID,CNT,APPNAME
0,1063609024,5/31/2022 3:50:54 PM,190959,1,779,1,Web
1,1064338610,7/2/2022 11:00:52 AM,190959,1,779,1,Mobile
2,1065032817,8/1/2022 1:04:24 PM,190959,1,780,1,Web
3,1065775681,9/2/2022 4:26:38 PM,190959,1,780,1,Mobile
4,1063648132,6/2/2022 12:47:48 PM,190962,2,87,1,Mobile
...,...,...,...,...,...,...,...
1692037,1064532375,7/9/2022 7:36:42 PM,375919,1,782,1,Mobile
1692038,1063790745,6/9/2022 8:04:51 AM,375921,1,780,1,Mobile
1692039,1064176352,6/26/2022 10:06:06 AM,375921,1,782,1,Mobile
1692040,1064804419,7/22/2022 9:29:28 AM,375921,1,782,1,Mobile


In [4]:
sales_after_df=pd.read_csv('Data/my_sales_after_14010624.csv' , encoding='utf_16_le')

In [9]:
sales_after_df

,TRANSACTION_ID,TRANSACTION_DATE,CUSOMER_ID,PRODUCT_TYPE,PRODUCTID,CNT,APPNAME
0,1066492863,10/2/2022 10:29:18 AM,190959,1,933,1,Mobile
1,1067065226,11/2/2022 2:53:23 PM,190959,1,933,1,Mobile
2,1066498962,10/2/2022 4:29:29 PM,190961,1,932,1,Web
3,1067188219,11/8/2022 9:33:35 PM,190961,1,932,1,Web
4,1066231924,9/21/2022 9:39:35 PM,190962,2,87,2,Mobile
...,...,...,...,...,...,...,...
1153792,1066561552,10/5/2022 9:44:44 PM,699959,2,87,1,Mobile
1153793,1066664884,10/11/2022 2:25:22 PM,699959,1,936,1,Mobile
1153794,1067222806,11/10/2022 2:58:11 PM,699959,1,516,1,Mobile
1153795,1066884679,10/23/2022 2:21:25 AM,699960,1,960,1,Web


In [5]:
#turn all column names to lower case
sales_before_df.columns=[col.lower() for col in sales_before_df.columns]
sales_after_df.columns=[col.lower() for col in sales_after_df.columns]

In [6]:
sales_before_df.dtypes

transaction_id       int64
transaction_date    object
cusomer_id           int64
product_type         int64
productid            int64
cnt                  int64
appname             object
dtype: object

In [7]:
# to convert date column from string to datetime
sales_before_df['transaction_date_formatted'] = pd.to_datetime(sales_before_df['transaction_date'], format='%m/%d/%Y %I:%M:%S %p' , errors='coerce')
sales_after_df['transaction_date_formatted'] = pd.to_datetime(sales_after_df['transaction_date'], format='%m/%d/%Y %I:%M:%S %p' , errors='coerce')



In [12]:
#dropping the old date columns
sales_before_df.drop('transaction_date' , axis=1 , inplace=True)
sales_after_df.drop('transaction_date' ,axis=1,  inplace=True)

In [16]:
sales_after_df.rename(columns={'transaction_date_formatted': 'transaction_date'} , inplace=True)
sales_after_df

,transaction_id,cusomer_id,product_type,productid,cnt,appname,transaction_date
0,1066492863,190959,1,933,1,Mobile,2022-10-02 10:29:18
1,1067065226,190959,1,933,1,Mobile,2022-11-02 14:53:23
2,1066498962,190961,1,932,1,Web,2022-10-02 16:29:29
3,1067188219,190961,1,932,1,Web,2022-11-08 21:33:35
4,1066231924,190962,2,87,2,Mobile,2022-09-21 21:39:35
...,...,...,...,...,...,...,...
1153792,1066561552,699959,2,87,1,Mobile,2022-10-05 21:44:44
1153793,1066664884,699959,1,936,1,Mobile,2022-10-11 14:25:22
1153794,1067222806,699959,1,516,1,Mobile,2022-11-10 14:58:11
1153795,1066884679,699960,1,960,1,Web,2022-10-23 02:21:25


In [17]:
sales_before_df.rename(columns={'transaction_date_formatted': 'transaction_date'} , inplace=True)
sales_before_df

,transaction_id,cusomer_id,product_type,productid,cnt,appname,transaction_date
0,1063609024,190959,1,779,1,Web,2022-05-31 15:50:54
1,1064338610,190959,1,779,1,Mobile,2022-07-02 11:00:52
2,1065032817,190959,1,780,1,Web,2022-08-01 13:04:24
3,1065775681,190959,1,780,1,Mobile,2022-09-02 16:26:38
4,1063648132,190962,2,87,1,Mobile,2022-06-02 12:47:48
...,...,...,...,...,...,...,...
1692037,1064532375,375919,1,782,1,Mobile,2022-07-09 19:36:42
1692038,1063790745,375921,1,780,1,Mobile,2022-06-09 08:04:51
1692039,1064176352,375921,1,782,1,Mobile,2022-06-26 10:06:06
1692040,1064804419,375921,1,782,1,Mobile,2022-07-22 09:29:28


In [ ]:
sql= get_sql_config()
sql

In [36]:
import psycopg2
# Update connection string information 


host = sql["host"]
dbname = "postgres"
user = "oskar"
password = sql['password']
sslmode = "require"
# Construct connection string

conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Drop previous table of same name if one exists


cursor.execute("DROP TABLE IF EXISTS inventory;")
print("Finished dropping table (if existed)")
# Create a table



OperationalError: could not connect to server: Operation timed out
	Is the server running on host "oskar.postgres.database.azure.com" (108.143.92.61) and accepting
	TCP/IP connections on port 5432?


In [33]:
conn=psycopg2.connect(user=f'{sql["user"]}' , password=f'{sql["password"]}' , host=f'{sql["host"]}', port=f'{sql["port"]}' , database='postgres' )

OperationalError: could not connect to server: Operation timed out
	Is the server running on host "oskar.postgres.database.azure.com" (108.143.92.61) and accepting
	TCP/IP connections on port 5432?


In [23]:
schema='capstone'
engine=sqlalchemy.create_engine(sql)

try:
    # Attempt to execute a simple query
    result = engine.execute("SELECT 1")
    
    # If the query executes successfully, the connection is established
    print("Connection successful!")
    
    # Fetch and print the result
    print(result.fetchone())
    
except Exception as e:
    # If an exception occurs, the connection is not successful
    print("Connection failed!")
    print(e)


AttributeError: 'dict' object has no attribute '_instantiate_plugins'

In [58]:
query='select * from DPARTMENT'
result = engine.execute(query)

OperationalError: (psycopg2.OperationalError) received invalid response to SSL negotiation: 

(Background on this error at: https://sqlalche.me/e/14/e3q8)